In [197]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [198]:
data_dict = {}
current_key = None
current_values = []

with open('Earthquakes_TRAIN.ts', 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('@'):
            if current_key is not None:
                if len(current_values) == 1:
                    data_dict[current_key] = current_values[0]
                else:
                    data_dict[current_key] = current_values
                current_values = []
            current_key = line
        else:
            current_values.append(line)

if current_key is not None:
    if len(current_values) == 1:
        data_dict[current_key] = current_values[0]
    else:
        data_dict[current_key] = current_values

In [199]:
split_data = [item.split(':') for item in data_dict['@data']]

In [200]:
split_data = [item.split(':') for item in data_dict['@data']]

pdfeatures0 = [list(map(float, item[0].split(','))) for item in split_data]
# pdfeatures1 = [list(map(float, item[1].split(','))) for item in split_data]
# pdfeatures2 = [list(map(float, item[2].split(','))) for item in split_data]
# pdfeatures3 = [list(map(float, item[3].split(','))) for item in split_data]
# pdfeatures4 = [list(map(float, item[4].split(','))) for item in split_data]
# pdfeatures5 = [list(map(float, item[5].split(','))) for item in split_data]
# pdfeatures6 = [list(map(float, item[6].split(','))) for item in split_data]
# pdfeatures7 = [list(map(float, item[7].split(','))) for item in split_data]
# pdfeatures8 = [list(map(float, item[8].split(','))) for item in split_data]
# pdfeatures9 = [list(map(float, item[9].split(','))) for item in split_data]
# pdfeatures10 = [list(map(float, item[10].split(','))) for item in split_data]
# pdfeatures11 = [list(map(float, item[11].split(','))) for item in split_data]
# pdfeatures12 = [list(map(float, item[12].split(','))) for item in split_data]
# pdfeatures13 = [list(map(float, item[13].split(','))) for item in split_data]

features0 = np.array([list(map(float, item[0].split(','))) for item in split_data])
# features1 = np.array([list(map(float, item[1].split(','))) for item in split_data])
# features2 = np.array([list(map(float, item[2].split(','))) for item in split_data])
# features3 = np.array([list(map(float, item[3].split(','))) for item in split_data])
# features4 = np.array([list(map(float, item[4].split(','))) for item in split_data])
# features5 = np.array([list(map(float, item[5].split(','))) for item in split_data])
# features6 = np.array([list(map(float, item[6].split(','))) for item in split_data])
# features7 = np.array([list(map(float, item[7].split(','))) for item in split_data])
# features8 = np.array([list(map(float, item[8].split(','))) for item in split_data])
# features9 = np.array([list(map(float, item[9].split(','))) for item in split_data])
# features10 = np.array([list(map(float, item[10].split(','))) for item in split_data])
# features11 = np.array([list(map(float, item[11].split(','))) for item in split_data])
# features12 = np.array([list(map(float, item[12].split(','))) for item in split_data])
# features13 = np.array([list(map(float, item[13].split(','))) for item in split_data])

labels = [0 if int(item[-1]) else 1 for item in split_data]

pddf = pd.DataFrame({'F1': pdfeatures0,
                  #  'F2': pdfeatures1,
                  #  'F3': pdfeatures2,
                  #  'F4': pdfeatures3,
                #    'F5': pdfeatures4,
                #    'F6': pdfeatures5,
                #    'F7': pdfeatures6,
                #    'F8': pdfeatures7,
                #    'F9': pdfeatures8,
                #    'F10': pdfeatures9,
                #    'F11': pdfeatures10,
                #    'F12': pdfeatures11,
                #    'F13': pdfeatures12,
                #    'F14': pdfeatures13,
                   'Labels': labels})
df = np.array(features0)#, features1, features2, features3], dtype=float)#, features4, features5, features6, features7, features8, features9, features10, features11, features12, features13], dtype=float)
labels = np.array(labels, dtype=int)

In [218]:
pd.DataFrame(labels[161:]).value_counts()

1    135
0     26
Name: count, dtype: int64

In [201]:
df_features = torch.tensor(df).float().unsqueeze(1)
df_labels = torch.tensor(labels).long()

In [202]:
df_train_features, df_train_labels, df_test_features, df_test_labels = df_features[:161,:], df_labels[:161], df_features[161:,:], df_labels[161:]

In [221]:
class LSTMModel(nn.Module):
    def __init__(self, input_d, hidden_d, layer_d, output_d):
        super(LSTMModel, self).__init__()
        
        self.hidden_dim = hidden_d
        self.layer_dim = layer_d

        # LSTM model
        self.lstm = nn.LSTM(input_d, hidden_d, layer_d, batch_first=True) 
        # batch_first=True (batch_dim, seq_dim, feature_dim)
        self.drp = nn.Dropout(0.7)
        self.fc = nn.Linear(hidden_d, output_d)
        self.relu = nn.ReLU(hidden_d)

    def forward(self, x):
    
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.drp(out)
        out = self.fc(out[:, -1, :]) 
        return self.relu(out)

In [222]:
model = LSTMModel(input_d=512, hidden_d=100, layer_d=100, output_d=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    outputs = model(df_train_features)
    loss = criterion(outputs.float(), df_train_labels.squeeze())
    loss.backward()
    optimizer.step()
    print(f'Training loss: {loss}')
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        outputs = model( df_test_features)
        _, predicted = torch.max(outputs.data, 1)
    accuracy = accuracy_score(df_test_labels, predicted)
    print(f'Validation Accuracy: {accuracy:.2f}')

TypeError: __init__() got an unexpected keyword argument 'return_sequences'